## Atualizando o tensorflow para a versão 2

In [0]:
#!pip install --upgrade tensorflow

https://stackoverflow.com/questions/58752538/tensorflow-batch-size-or-steps-is-required-for-tensor-or-numpy-input-da

### Import the revelant packages

In [0]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [0]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [0]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [0]:
num_validation_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)
num_test_samples = tf.cast(mnist_info.splits['test'].num_examples, tf.int64)

In [0]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [0]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

#### Outline the model

In [0]:
#input_size = 784
hidden_layer_size = 100
output_size = 10

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
])

## Choice the optimizer and the loss function

##### Loss functions:
    binary_crossentropy
    categorical_crossentropy
    sparse_categorical_crossentropy

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

#### O que acontece dentro de uma época:
    1. No início de cada época, a perda de treinamento será definida como 0.
    2. O algoritmo irá iterar sobre um número predefinido de lotes, todos de train_data.
    3. Os pesos e vieses serão atualizados por quantas vezes houverem lotes.
    4. Obteremos um valor para a função de perda, indicando como está indo o treinamento.
    5. Veremos também uma precisão do treinamento.
    6. No final da época, o algoritmo irá propagar todo o conjunto completo de validação
    7. Quando atingirmos o número máximo de épocas, o treinamento terminará.

In [10]:
NUM_EPOCHS = 5

model.fit(
    train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2, validation_steps=10
)

Epoch 1/5
540/540 - 16s - loss: 0.3406 - accuracy: 0.9040 - val_loss: 0.1709 - val_accuracy: 0.9480
Epoch 2/5
540/540 - 15s - loss: 0.1403 - accuracy: 0.9586 - val_loss: 0.1193 - val_accuracy: 0.9650
Epoch 3/5
540/540 - 16s - loss: 0.0988 - accuracy: 0.9701 - val_loss: 0.0927 - val_accuracy: 0.9702
Epoch 4/5
540/540 - 16s - loss: 0.0777 - accuracy: 0.9764 - val_loss: 0.0758 - val_accuracy: 0.9760
Epoch 5/5
540/540 - 16s - loss: 0.0621 - accuracy: 0.9810 - val_loss: 0.0666 - val_accuracy: 0.9802


### Test the model

In [11]:
test_loss, test_accucacy = model.evaluate(test_data)

      1/Unknown - 3s 3s/step - loss: 0.0831 - accuracy: 0.9744